In [1]:
# process matlab produced csv and convert pickle back to csv
# Conny Lin | June 5, 2020

In [2]:
# load library
import os
import sys
import pickle
import time
import numpy as np
import pandas as pd
import subprocess
import glob

# import functions
dir_function_collection = [
'/Users/connylin/Dropbox/Code/language/python_lib/localpackage/mwt',
'/Users/connylin/Dropbox/CA/ED _20200119 Brain Station Data Science Diploma/Capstone/brainstation_capstone/0_lib'
]
for p in dir_function_collection:
    sys.path.insert(1, p)

import BrainStationLib as bs

In [3]:
# define global varialbes/paths
pDropbox_home = '/Users/connylin/Dropbox/MWT/db'
pCapstone = '/Users/connylin/Dropbox/CA/ED _20200119 Brain Station Data Science Diploma/Capstone/data'

In [4]:
# load MWTDB for dropbox
MWTDB = pd.read_csv(pDropbox_home+'/MWTDB.csv', index_col=False)
MWTDB

,mwtid,mwtpath,expname,groupname,platename
0,0,/Users/connylin/Dropbox/MWT/db/MWT/20141216B_S...,20141216B_SI_100s30x10s10s_MT1,RB982,20141216_175807
1,1,/Users/connylin/Dropbox/MWT/db/MWT/20141216B_S...,20141216B_SI_100s30x10s10s_MT1,RB982,20141216_173721
2,2,/Users/connylin/Dropbox/MWT/db/MWT/20141216B_S...,20141216B_SI_100s30x10s10s_MT1,RB982,20141216_182015
3,3,/Users/connylin/Dropbox/MWT/db/MWT/20141216B_S...,20141216B_SI_100s30x10s10s_MT1,RB982,20141216_171635
4,4,/Users/connylin/Dropbox/MWT/db/MWT/20141216B_S...,20141216B_SI_100s30x10s10s_MT1,RB982_400mM,20141216_180658
...,...,...,...,...,...
7289,7289,/Users/connylin/Dropbox/MWT/db/MWT to check/20...,20140819C_SM_100s30x10s10s,ungroup,20140819_172227
7290,7290,/Users/connylin/Dropbox/MWT/db/MWT to check/20...,20140819C_SM_100s30x10s10s,ungroup,20140819_191228
7291,7291,/Users/connylin/Dropbox/MWT/db/MWT to check/20...,20140819C_SM_100s30x10s10s,ungroup,20140819_182649
7292,7292,/Users/connylin/Dropbox/MWT/db/MWT to check/20...,20140819C_SM_100s30x10s10s,ungroup,20140819_173055


In [5]:
MWTDBTNT = pickle.load(open(os.path.join(pCapstone,'MWTDB_trinity_N2400mM.pickle'),'rb'))

In [6]:
istart = 300
for iMWTDB in MWTDB.index.values[istart:]:
    # get path
    pmwt = MWTDB.loc[iMWTDB, 'mwtpath']
    print(f'{iMWTDB}: {pmwt}')

    # check if has trinity_cleaned.csv, 
    ptrinity_cleaned = os.path.join(pmwt,'trinity_cleaned.csv')
    ptrinity_pickle = os.path.join(pmwt,'trinity_all_worms.pickle')
    ptrinity_idin = os.path.join(pmwt,'trinity_idin.csv')
    ptrinity_meta = os.path.join(pmwt,'trinity_meta.csv')
    ptrinity_csv = os.path.join(pmwt,'trinity.csv')
    
    # instantiate dataframe
    df = pd.DataFrame({'failed':[0]})

    # if no, trinity_cleaned.csv
    if not os.path.isfile(ptrinity_cleaned):
        print('\tno trinity_cleaned')
        # check if has trinity_all_worms.pickle
        # if yes
        if os.path.isfile(ptrinity_pickle):
            print(f'\tfound pickle file')
            # load data
            df = pickle.load(open(ptrinity_pickle,'rb'))
            # get MWTDB id
            plateid = MWTDB.loc[iMWTDB, 'mwtid']
            # add mwtid column
            df.insert(0, 'mwtid', np.tile(plateid, df.shape[0]))
            # drop mwtid_trdb column
            df.drop(columns=['mwtid_trdb'], inplace=True)
            # save as trinity_cleaned.csv
            df.to_csv(ptrinity_cleaned, index=False)
            print(f'\twrite trinity_cleaned.csv')
            
        # if no, 
        elif not os.path.isfile(ptrinity_pickle):
            print('\tno pickle')
            # check if have trinity_idin.csv
            # if no, 
            if not os.path.isfile(ptrinity_idin):
                print(f'\tno trinity_idin.csv')
                # check if has both trinity_meta.csv and trinity.csv
                # if yes, 
                if os.path.isfile(ptrinity_meta) & \
                    os.path.isfile(ptrinity_csv):
                    print('\thas both meta and csv')
                    assert False, 'code this option'
                    # load trinity_meta.csv
                    # create worm id column by rows
                    # load trinity.csv
                    # concat with trinity.csv, 
                    # remove columns
                # if no, leave the plate be
                else:
                    print(f'\tno both (meta, csv) or pickle. Skip')
                    continue
                    assert False, 'code this option'
                    
            # if yes,
            elif os.path.isfile(ptrinity_idin):
                print(f'\tfound trinity_idin.csv')
                # load trinity_idin.csv
                df = pd.read_csv(ptrinity_idin)
                TD = bs.TrinityData(ptrinity_idin)
                df = TD.loaddata()
            else:
                print(f'\tsome weird unforseen scenario')
                assert False, 'code this option'
            
            # [now should have the same dataframe]
            # check df is right size
            if df.shape[1]==14:
                print(f'\tgathered right dataframe.csv')
                # clean out nan values
                df.dropna(axis=0, inplace=True)
                # add plate id
                plateid = MWTDB.loc[iMWTDB, 'mwtid']
                df.insert(0, 'mwtid', np.tile(plateid, df.shape[0]))
                # add etoh id
                etoh = 0
                if '00mM' in MWTDB.loc[iMWTDB, 'groupname']:
                    etoh = 1
                df.insert(1, 'etoh', np.tile(etoh, df.shape[0]))
                # save as trinity_cleaned.csv 
                print(f'\twrite trinity_cleaned.csv')
                df.to_csv(ptrinity_cleaned, index=False)

        else:
            print(f'\tsome weird unforseen scenario')
            assert False, 'code this option'
    else:
        print(f'\talready has trinity_cleaned.csv')

    # final check to see if trinity_cleaned exist
    if os.path.isfile(ptrinity_cleaned):
        print(f'\ttrinity_cleaned.csv present, delete redundant files')
        # if so, delete trinity_meta.csv, trinity.csv, and pickle
        if os.path.isfile(ptrinity_idin):
            os.remove(ptrinity_idin)
            print('\tdeleted trinity_idin.csv')
        if os.path.isfile(ptrinity_pickle):
            os.remove(ptrinity_pickle)
            print('\tdeleted trinity pickle file')
        if os.path.isfile(ptrinity_meta):
            os.remove(ptrinity_meta)
            print('\tdeleted trinity meta file')
        if os.path.isfile(ptrinity_csv):
            os.remove(ptrinity_csv)
            print('\tdeleted trinity.csv file')

300: /Users/connylin/Dropbox/MWT/db/MWT/20130926B_SJ_100s30x10s10s_Neuroligin/VC228_400mM/20130926_205708
	already has trinity_cleaned.csv
	trinity_cleaned.csv present, delete redundant files
301: /Users/connylin/Dropbox/MWT/db/MWT/20130926B_SJ_100s30x10s10s_Neuroligin/VC228_400mM/20130926_210428
	already has trinity_cleaned.csv
	trinity_cleaned.csv present, delete redundant files
302: /Users/connylin/Dropbox/MWT/db/MWT/20130926B_SJ_100s30x10s10s_Neuroligin/VC228_400mM/20130926_204929
	already has trinity_cleaned.csv
	trinity_cleaned.csv present, delete redundant files
303: /Users/connylin/Dropbox/MWT/db/MWT/20130926B_SJ_100s30x10s10s_Neuroligin/N2_400mM/20130926_185424
	already has trinity_cleaned.csv
	trinity_cleaned.csv present, delete redundant files
304: /Users/connylin/Dropbox/MWT/db/MWT/20130926B_SJ_100s30x10s10s_Neuroligin/N2_400mM/20130926_183902
	already has trinity_cleaned.csv
	trinity_cleaned.csv present, delete redundant files
305: /Users/connylin/Dropbox/MWT/db/MWT/201309

KeyboardInterrupt: 

In [ ]:
# stopped at 2356

In [ ]:
assert False, 'stop'